## Set Up

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
!pip install openpyxl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.9/250.9 kB 18.6 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


## Extraction

In [2]:
data_source = _dntk.execute_sql(
  'SELECT *\nFROM st_read(\'spotify_data_songs_.xlsx\')',
  'SQL_DEEPNOTE_DATAFRAME_SQL',
  audit_sql_comment='',
  sql_cache_mode='cache_disabled',
  return_variable_type='dataframe'
)
data_source

,spotify_track_uri,ts,platform,ms_played,track_name,artist_name,album_name,reason_start,reason_end,shuffle,skipped
0,2J3n32GeLmMjwuAzyhcSNe,2013-07-08 02:44:34,web player,3185,"Say It, Just Say It",The Mowgli's,Waiting For The Dawn,autoplay,clickrow,FALSE,FALSE
1,1oHxIPqJyvAYHy0PVrDU98,2013-07-08 02:45:37,web player,61865,Drinking from the Bottle (feat. Tinie Tempah),Calvin Harris,18 Months,clickrow,clickrow,FALSE,FALSE
2,487OPlneJNni3NWC8SYqhW,2013-07-08 02:50:24,web player,285386,Born To Die,Lana Del Rey,Born To Die - The Paradise Edition,clickrow,unknown,FALSE,FALSE
3,5IyblF777jLZj1vGHG2UD3,2013-07-08 02:52:40,web player,134022,Off To The Races,Lana Del Rey,Born To Die - The Paradise Edition,trackdone,clickrow,FALSE,FALSE
4,0GgAAB0ZMllFhbNc3mAodO,2013-07-08 03:17:52,web player,0,Half Mast,Empire Of The Sun,Walking On A Dream,clickrow,nextbtn,FALSE,FALSE
...,...,...,...,...,...,...,...,...,...,...,...
149855,4Fz1WWr5o0OrlIcZxcyZtK,2024-12-15 23:06:19,android,1247,On The Way Home,John Mayer,Paradise Valley,fwdbtn,fwdbtn,TRUE,TRUE
149856,0qHMhBZqYb99yhX9BHcIkV,2024-12-15 23:06:21,android,1515,Magical Mystery Tour - Remastered 2009,The Beatles,Magical Mystery Tour,fwdbtn,fwdbtn,TRUE,TRUE
149857,0HHdujGjOZChTrl8lJWEIq,2024-12-15 23:06:22,android,1283,"Stop This Train - Live at the Nokia Theatre, L...",John Mayer,Where the Light Is: John Mayer Live In Los Ang...,fwdbtn,fwdbtn,TRUE,TRUE
149858,7peh6LUcdNPcMdrSH4JPsM,2024-12-15 23:06:23,android,1306,I Don't Trust Myself (With Loving You),John Mayer,Continuum,fwdbtn,fwdbtn,TRUE,TRUE


## Transformation

### Main query

In [3]:
spotify = _dntk.execute_sql(
  'select \n    spotify_track_uri\n    ,cast(ts as timestamp) as ts\n    ,platform\n    ,ms_played\n    ,track_name\n    ,artist_name\n    ,album_name\n    ,reason_start\n    ,reason_end\n    ,shuffle\n    ,skipped\n    ,round(ms_played / 1000 / 60, 1) as minutes_played\n    ,case \n        when skipped = \'TRUE\' then 1\n        when skipped = \'FALSE\' then 0 end as skipped_flag\n    ,lag(ts) over(partition by artist_name order by ts) as previous_ts_date\nfrom data_source\n',
  'SQL_DEEPNOTE_DATAFRAME_SQL',
  audit_sql_comment='',
  sql_cache_mode='cache_disabled',
  return_variable_type='dataframe'
)
spotify

,spotify_track_uri,ts,platform,ms_played,track_name,artist_name,album_name,reason_start,reason_end,shuffle,skipped,minutes_played,skipped_flag,previous_ts_date
0,06xNkl12TrxwGYrwTGlUUd,2022-02-22 01:54:29,android,169460,Main Title (The Notebook),Aaron Zigman,The Notebook (Original Motion Picture Soundtrack),trackdone,trackdone,FALSE,FALSE,2.8,0.0,NaT
1,0UmR5k7YxioGuU6V0XVPbZ,2023-01-02 20:43:23,android,184977,Blue Something,Agustín Amigó,Blue Something,trackdone,trackdone,TRUE,FALSE,3.1,0.0,NaT
2,0LbZg654PnwFu7lcj5aPtb,2023-01-03 21:44:32,android,193727,Manzanilla,Agustín Amigó,Swatches,trackdone,trackdone,TRUE,FALSE,3.2,0.0,2023-01-02 20:43:23
3,2jsQWRYISLjXA2x3LQMzH6,2023-01-24 18:34:17,android,168000,Days of Rain,Agustín Amigó,Days of Rain,trackdone,trackdone,TRUE,FALSE,2.8,0.0,2023-01-03 21:44:32
4,0LbZg654PnwFu7lcj5aPtb,2023-01-25 18:23:23,android,193727,Manzanilla,Agustín Amigó,Swatches,trackdone,trackdone,TRUE,FALSE,3.2,0.0,2023-01-24 18:34:17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149855,27ebni0DfbT5Owz6W42HP8,2021-09-25 06:40:55,android,241560,Nunca Me Olvides,Yandel,Dangerous,trackdone,trackdone,TRUE,FALSE,4.0,0.0,2020-12-20 08:25:59
149856,1UulFvlRZPoCWwUBzk5ImI,2021-12-23 07:57:52,android,213520,Jaque Mate,Yandel,Legacy - De Líder a Leyenda Tour,trackdone,trackdone,FALSE,FALSE,3.6,0.0,2021-09-25 06:40:55
149857,2oiixB9QMIzhWaHGVlQx4g,2023-04-15 21:41:01,android,216148,Yandel 150,Yandel,Yandel 150,clickrow,trackdone,TRUE,FALSE,3.6,0.0,2021-12-23 07:57:52
149858,4FAKtPVycI4DxoOHC01YqD,2023-05-21 07:23:39,android,77915,Yandel 150,Yandel,Resistencia,trackdone,fwdbtn,FALSE,TRUE,1.3,1.0,2023-04-15 21:41:01


### Business Question #1 (Artist Summary)

***Which artists are listened to the most, and what is the engagement level with their music, measured by total minutes and the number of times their songs are skipped?***

In [4]:
_dntk.execute_sql(
  'select \n    artist_name\n    ,count(*) as total_reproductions\n    ,sum(minutes_played) as total_minutes_played\n    ,sum(skipped_flag) as total_skips\nfrom spotify\ngroup by artist_name\nhaving count(*) > 1\norder by sum(minutes_played) desc',
  'SQL_DEEPNOTE_DATAFRAME_SQL',
  audit_sql_comment='',
  sql_cache_mode='cache_disabled',
  return_variable_type='dataframe'
)

,artist_name,total_reproductions,total_minutes_played,total_skips
0,The Beatles,13621,20079.4,388.0
1,The Killers,6878,17615.5,197.0
2,John Mayer,4855,12084.3,153.0
3,Bob Dylan,3814,9479.7,163.0
4,Paul McCartney,2697,5940.2,107.0
...,...,...,...,...
2259,Mariachi México de Pepe Villa,2,0.0,0.0
2260,The Wanted,2,0.0,2.0
2261,Joe Lovano,2,0.0,2.0
2262,Foreigner,2,0.0,2.0


### Business Question #2 (Time Between Listens)

***What is the listening cadence or frequency for each artist, measured by the time elapsed between consecutive plays?***

In [5]:
_dntk.execute_sql(
  'select \n    artist_name\n    ,track_name\n    ,ts\n    ,previous_ts_date\n    ,round(JULIAN(ts) - JULIAN(previous_ts_date),2) as days_since_previous_play\n    ,ts - previous_ts_date as days_hours_since_previous_play\nfrom spotify',
  'SQL_DEEPNOTE_DATAFRAME_SQL',
  audit_sql_comment='',
  sql_cache_mode='cache_disabled',
  return_variable_type='dataframe'
)

,artist_name,track_name,ts,previous_ts_date,days_since_previous_play,days_hours_since_previous_play
0,Aaron Zigman,Main Title (The Notebook),2022-02-22 01:54:29,NaT,NaN,NaT
1,Agustín Amigó,Blue Something,2023-01-02 20:43:23,NaT,NaN,NaT
2,Agustín Amigó,Manzanilla,2023-01-03 21:44:32,2023-01-02 20:43:23,1.04,1 days 01:01:09
3,Agustín Amigó,Days of Rain,2023-01-24 18:34:17,2023-01-03 21:44:32,20.87,20 days 20:49:45
4,Agustín Amigó,Manzanilla,2023-01-25 18:23:23,2023-01-24 18:34:17,0.99,0 days 23:49:06
...,...,...,...,...,...,...
149855,Yandel,Nunca Me Olvides,2021-09-25 06:40:55,2020-12-20 08:25:59,278.93,278 days 22:14:56
149856,Yandel,Jaque Mate,2021-12-23 07:57:52,2021-09-25 06:40:55,89.05,89 days 01:16:57
149857,Yandel,Yandel 150,2023-04-15 21:41:01,2021-12-23 07:57:52,478.57,478 days 13:43:09
149858,Yandel,Yandel 150,2023-05-21 07:23:39,2023-04-15 21:41:01,35.40,35 days 09:42:38


### Business Question #3

***What is the proportion of active listening (user-initiated plays) versus passive listening (autoplay)?***

Calculating the number of tracks started by a direct user action versus those started automatically after another track finished.

In [6]:
df_3 = _dntk.execute_sql(
  'with count_per_start_type as (\nselect \n    track_name\n    ,case \n        when reason_start = \'trackdone\' then \'Autoplay\'\n        when reason_start in(\'fwdbtn\', \'backbtn\', \'playbtn\', \'nextbtn\', \'remote\', \'popup\') then \'user_action\'\n        when reason_start in(\'appload\',\'unknown\') then \'unknown\'\n        else \'other\' end as start_type\nfrom spotify\n)\n\nselect \n    start_type\n    ,count(track_name) as start_type_count\nfrom count_per_start_type\ngroup by start_type\norder by count(track_name) desc',
  'SQL_DEEPNOTE_DATAFRAME_SQL',
  audit_sql_comment='',
  sql_cache_mode='cache_disabled',
  return_variable_type='dataframe'
)
df_3

,start_type,start_type_count
0,Autoplay,76491
1,user_action,57853
2,other,11772
3,unknown,3744


### Business Question #4

***For a given artist, which tracks are listened to for longer than that artist's average listening time?***

In [7]:
_dntk.execute_sql(
  'with avg_minutes as (\n    select \n    artist_name\n    ,track_name\n    ,minutes_played\n    ,avg(minutes_played) over(partition by artist_name) as artist_avg_minutes\nfrom spotify\n    )\n\nselect \n    artist_name\n    ,track_name\n    ,minutes_played\n    ,artist_avg_minutes\n    ,case\n        when minutes_played > artist_avg_minutes then \'above average\'\n        when minutes_played < artist_avg_minutes then \'below average\'\n        else \'Equal to average\' end as comparison\nfrom avg_minutes',
  'SQL_DEEPNOTE_DATAFRAME_SQL',
  audit_sql_comment='',
  sql_cache_mode='cache_disabled',
  return_variable_type='dataframe'
)

,artist_name,track_name,minutes_played,artist_avg_minutes,comparison
0,Aaron Zigman,Main Title (The Notebook),2.8,2.8,Equal to average
1,Agustín Amigó,Blue Something,3.1,3.1,Equal to average
2,Agustín Amigó,Manzanilla,3.2,3.1,above average
3,Agustín Amigó,Days of Rain,2.8,3.1,below average
4,Agustín Amigó,Manzanilla,3.2,3.1,above average
...,...,...,...,...,...
149855,Yandel,Nunca Me Olvides,4.0,2.0,above average
149856,Yandel,Jaque Mate,3.6,2.0,above average
149857,Yandel,Yandel 150,3.6,2.0,above average
149858,Yandel,Yandel 150,1.3,2.0,below average


### Business Question #5:

***How do the reasons for starting a track compare to the reasons for ending it, in a single, unified view?***

In [8]:
_dntk.execute_sql(
  'with start_reasons as (\nselect \n    \'start_reason\' as category_type\n    ,reason_start as reason\n    ,count(track_name) as track_count\nfrom spotify\ngroup by category_type, reason_start\n),\n\nend_reasons as (\nselect \n    \'end_reason\' as category_type\n    ,reason_end as reason\n    ,count(track_name) as track_count\nfrom spotify\ngroup by category_type, reason_end\n)\n\nselect *\nfrom start_reasons\nunion all\nselect * \nfrom end_reasons ',
  'SQL_DEEPNOTE_DATAFRAME_SQL',
  audit_sql_comment='',
  sql_cache_mode='cache_disabled',
  return_variable_type='dataframe'
)

,category_type,reason,track_count
0,start_reason,fwdbtn,53692
1,start_reason,trackdone,76491
2,start_reason,backbtn,2202
3,start_reason,playbtn,1456
4,start_reason,appload,3721
5,start_reason,unknown,23
6,start_reason,remote,477
7,start_reason,None,452
8,start_reason,nextbtn,21
9,start_reason,popup,5


### Business Question #5:

***In which part of the day is the listening time per track longer, considering only periods with significant listening?***

In [9]:
_dntk.execute_sql(
  'with hour_day as (\nselect \n    extract(hour from ts) as hour_day\n    ,track_name\n    ,minutes_played\n    ,case \n        when extract(hour from ts) between 6 and 11 then \'Morning\'\n        when extract(hour from ts) between 12 and 17 then \'Afternoon\'\n        when extract(hour from ts) between 18 and 22 then \'Evening\'\n        else \'night\' end as time_of_day\nfrom spotify\n)\n\nselect \n    time_of_day\n    ,count(track_name) as track_count\n    ,round(avg(minutes_played),2) as avg_minutes_played\nfrom hour_day \ngroup by time_of_day\nhaving avg(minutes_played) > 1.5\norder by avg(minutes_played) desc',
  'SQL_DEEPNOTE_DATAFRAME_SQL',
  audit_sql_comment='',
  sql_cache_mode='cache_disabled',
  return_variable_type='dataframe'
)

,time_of_day,track_count,avg_minutes_played
0,Afternoon,24797,2.47
1,Evening,45272,2.23
2,night,61893,1.99
3,Morning,17898,1.94


In [10]:
spotify

,spotify_track_uri,ts,platform,ms_played,track_name,artist_name,album_name,reason_start,reason_end,shuffle,skipped,minutes_played,skipped_flag,previous_ts_date
0,06xNkl12TrxwGYrwTGlUUd,2022-02-22 01:54:29,android,169460,Main Title (The Notebook),Aaron Zigman,The Notebook (Original Motion Picture Soundtrack),trackdone,trackdone,FALSE,FALSE,2.8,0.0,NaT
1,0UmR5k7YxioGuU6V0XVPbZ,2023-01-02 20:43:23,android,184977,Blue Something,Agustín Amigó,Blue Something,trackdone,trackdone,TRUE,FALSE,3.1,0.0,NaT
2,0LbZg654PnwFu7lcj5aPtb,2023-01-03 21:44:32,android,193727,Manzanilla,Agustín Amigó,Swatches,trackdone,trackdone,TRUE,FALSE,3.2,0.0,2023-01-02 20:43:23
3,2jsQWRYISLjXA2x3LQMzH6,2023-01-24 18:34:17,android,168000,Days of Rain,Agustín Amigó,Days of Rain,trackdone,trackdone,TRUE,FALSE,2.8,0.0,2023-01-03 21:44:32
4,0LbZg654PnwFu7lcj5aPtb,2023-01-25 18:23:23,android,193727,Manzanilla,Agustín Amigó,Swatches,trackdone,trackdone,TRUE,FALSE,3.2,0.0,2023-01-24 18:34:17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149855,27ebni0DfbT5Owz6W42HP8,2021-09-25 06:40:55,android,241560,Nunca Me Olvides,Yandel,Dangerous,trackdone,trackdone,TRUE,FALSE,4.0,0.0,2020-12-20 08:25:59
149856,1UulFvlRZPoCWwUBzk5ImI,2021-12-23 07:57:52,android,213520,Jaque Mate,Yandel,Legacy - De Líder a Leyenda Tour,trackdone,trackdone,FALSE,FALSE,3.6,0.0,2021-09-25 06:40:55
149857,2oiixB9QMIzhWaHGVlQx4g,2023-04-15 21:41:01,android,216148,Yandel 150,Yandel,Yandel 150,clickrow,trackdone,TRUE,FALSE,3.6,0.0,2021-12-23 07:57:52
149858,4FAKtPVycI4DxoOHC01YqD,2023-05-21 07:23:39,android,77915,Yandel 150,Yandel,Resistencia,trackdone,fwdbtn,FALSE,TRUE,1.3,1.0,2023-04-15 21:41:01


## Exploratory Data Analysys (Python)

### Business Question #1:

***In which part of the day is the listening time per track longer, considering only periods with significant listening?***

In [11]:
spotify['ts'] = pd.to_datetime(spotify['ts'])
spotify['hour'] = spotify['ts'].dt.hour
conditions = [
    (spotify['hour'] >= 6) & (spotify['hour'] <= 11),
    (spotify['hour'] >= 12) & (spotify['hour'] <= 17),
    (spotify['hour'] >= 18) & (spotify['hour'] <= 22)
]
choices = [
    'Morning', 'Afternoon','Evening'
]
spotify['time_of_day'] = np.select(conditions, choices, default='Night')

time_day_grouped = spotify.groupby('time_of_day')[['track_name','minutes_played']].agg({'track_name':'count','minutes_played':'mean'}).round(2).reset_index()
time_day_grouped = time_day_grouped.rename(columns ={'track_name':'track_count','minutes_played':'avg_minutes_played'})
time_day_grouped_filtered = time_day_grouped.loc[(time_day_grouped['time_of_day'] != 'Night') & (time_day_grouped['avg_minutes_played'] > 1.5)].sort_values('avg_minutes_played', ascending = False)
time_day_grouped_filtered

,time_of_day,track_count,avg_minutes_played
0,Afternoon,24797,2.47
1,Evening,45272,2.23
2,Morning,17898,1.94


In [12]:
fig = px.bar(time_day_grouped,x='time_of_day',y='avg_minutes_played',)
fig.show()

In [13]:
fig = px.box(spotify,x='time_of_day',y='minutes_played')
fig.show()

In [14]:
spotify

,spotify_track_uri,ts,platform,ms_played,track_name,artist_name,album_name,reason_start,reason_end,shuffle,skipped,minutes_played,skipped_flag,previous_ts_date,hour,time_of_day
0,06xNkl12TrxwGYrwTGlUUd,2022-02-22 01:54:29,android,169460,Main Title (The Notebook),Aaron Zigman,The Notebook (Original Motion Picture Soundtrack),trackdone,trackdone,FALSE,FALSE,2.8,0.0,NaT,1,Night
1,0UmR5k7YxioGuU6V0XVPbZ,2023-01-02 20:43:23,android,184977,Blue Something,Agustín Amigó,Blue Something,trackdone,trackdone,TRUE,FALSE,3.1,0.0,NaT,20,Evening
2,0LbZg654PnwFu7lcj5aPtb,2023-01-03 21:44:32,android,193727,Manzanilla,Agustín Amigó,Swatches,trackdone,trackdone,TRUE,FALSE,3.2,0.0,2023-01-02 20:43:23,21,Evening
3,2jsQWRYISLjXA2x3LQMzH6,2023-01-24 18:34:17,android,168000,Days of Rain,Agustín Amigó,Days of Rain,trackdone,trackdone,TRUE,FALSE,2.8,0.0,2023-01-03 21:44:32,18,Evening
4,0LbZg654PnwFu7lcj5aPtb,2023-01-25 18:23:23,android,193727,Manzanilla,Agustín Amigó,Swatches,trackdone,trackdone,TRUE,FALSE,3.2,0.0,2023-01-24 18:34:17,18,Evening
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149855,27ebni0DfbT5Owz6W42HP8,2021-09-25 06:40:55,android,241560,Nunca Me Olvides,Yandel,Dangerous,trackdone,trackdone,TRUE,FALSE,4.0,0.0,2020-12-20 08:25:59,6,Morning
149856,1UulFvlRZPoCWwUBzk5ImI,2021-12-23 07:57:52,android,213520,Jaque Mate,Yandel,Legacy - De Líder a Leyenda Tour,trackdone,trackdone,FALSE,FALSE,3.6,0.0,2021-09-25 06:40:55,7,Morning
149857,2oiixB9QMIzhWaHGVlQx4g,2023-04-15 21:41:01,android,216148,Yandel 150,Yandel,Yandel 150,clickrow,trackdone,TRUE,FALSE,3.6,0.0,2021-12-23 07:57:52,21,Evening
149858,4FAKtPVycI4DxoOHC01YqD,2023-05-21 07:23:39,android,77915,Yandel 150,Yandel,Resistencia,trackdone,fwdbtn,FALSE,TRUE,1.3,1.0,2023-04-15 21:41:01,7,Morning


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=c69c347e-adeb-472b-b52c-7c0ad12c6123' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>